In [73]:

#1.  Read in data, flatten images, separate train (80%) and test (20%), generate labels
#2.  Find Covariance matrix of train data = AA_t
#3.  Do eigendecomposition of Covariance matrix
#4.  find eigenvectors that are responsible for 90% of the variance
#5.  

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re

X = []
y = []


for f in glob.iglob('data/**/**'):
    X.append(np.array(cv2.imread(f, cv2.IMREAD_GRAYSCALE)).flatten().tolist())
    index = int(f.split("\\")[-2].split("s")[1])
    y.append(index)


data_set = [(X[i], y[i]) for i in range(len(y))]

data_set.sort(key= lambda x: x[1])

train = []
test = []

for i, item in enumerate(data_set):
    if i % 10 > 7:
        test.append(item)
    else:
        train.append(item)

train_X = np.array([[X[0], X[1]] for X in train])[:, 0]
train_y = np.array([[X[0], X[1]] for X in train])[:, 1]

test_X = np.array([[X[0], X[1]] for X in test])[:, 0]
test_y = np.array([[X[0], X[1]] for X in test])[:, 1]

#Cov matrix of train data

# cov_mat = train_X.T @ train_X


c:\Users\sayba\anaconda3\lib\site-packages\ipykernel_launcher.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
c:\Users\sayba\anaconda3\lib\site-packages\ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
c:\Users\sayba\anaconda3\lib\site-packages\ipykernel_launcher.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarra

In [76]:
g = np.zeros((3,3))

In [77]:
g

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])